#Lab 1: Regression: Linear and Logistic

# **About this Lab:**

Throughout this lab, there are Graded Activities where you will be tasked with implementing critical blocks of code and submitting them for grading. The detail marking guide can be found on Moodle. This lab is 1% of total course weight

## Part -1 Linear Regression

Linear Regression is a statistical approach of modelling the realtionship between a scalar response (aka. dependent variable) and one or more explanatory variables (aka independent variables). Linear Regression is one of the most basic building block in Machine Learning algorithms.

In most of the python libraries, linear regression model is available as a blackbox. However, it is imperative to understand what goes on under the hood in order to have better grasp of algorithm.

The first part of this tutorial will focus on implementing vanilla linear regression from scratch using numpy and pandas.

Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### Getting Familiar with the Data

Add characteristics of Data from: https://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf

In [2]:
def load_diabetes_data():
    diabetes = datasets.load_diabetes()
    X, Y = diabetes['data'], diabetes['target'].reshape(len(diabetes['target']), 1)
    return diabetes, X, Y

def get_scale_object(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler

def get_scaled_data(scaler, data):
    return scaler.transform(data)

In [3]:
diabetes, X, Y = load_diabetes_data()
scaler = get_scale_object(X)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

m_train = X_train.shape[0]
m_test = X_test.shape[0]
n = X_train.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Number of features for each instance: num_features = " + str(n))
print("Features Names ", diabetes.feature_names)
print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(Y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(Y_test.shape))
# scaler = get_scale_object(raw_X)
# X = get_scaled_data(scaler, raw_X)

Number of training examples: m_train = 353
Number of testing examples: m_test = 89
Number of features for each instance: num_features = 10
Features Names  ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
train_set_x shape: (353, 10)
train_set_y shape: (353, 1)
test_set_x shape: (89, 10)
test_set_y shape: (89, 1)


### General Architecture of the Linear Regression

### Building Parts:
1. Initialise Model Parameters
2. Loop:

  a. Forward Propagation

  b. Cost Function

  c. Gradient Descent

#### Initialise Model Parameters

In [5]:
def initialize_dimensions(X):
    m, n = X.shape
    return m, n

def initialize_weight_vectors(dim):
    W = np.zeros(shape = dim)
    b = 0
    return W, b

m_train, n = initialize_dimensions(X_train)
W, b = initialize_weight_vectors((1, n))
W.shape, b

((1, 10), 0)

#### Forward Propagation and Cost Function


In [6]:
def compute_regression_cost(y, y_pred, m):
    cost = (1.0/(2*m)) * (np.sum(y_pred - y)**2)
    return cost

def compute_regression_gradients(y_pred, y, X, m):
    dw = (1.0/m)* np.dot((y_pred - y).T, X)
    db = (1.0/m) * np.sum((y_pred - y))
    return dw, db

def propagation(w, b, X, Y):
    # FORWARD PROPAGATION (FROM X TO COST)

    z = (np.dot(X,w.T) + b)
    cost = compute_regression_cost(Y, z, X.shape[1])

    #END YOUR CODE

    # BACKWARD PROPAGATION (TO FIND GRAD)

    dw, db = compute_regression_gradients(z, Y, X, X.shape[0])

    cost = np.squeeze(cost)

    grads = {"dw": dw,
             "db": db}
    #END YOUR CODE
    return grads, cost

### Gradient Descent Optimization

In [7]:
def train_linear_regression(X, Y, learning_rate = 0.2, n_epochs = 1000, print_cost = True):
    m, n = initialize_dimensions(X)
    print(f"There are {n} features and {m} training examples")
    w, b = initialize_weight_vectors((1, n))
    print(f"W has a shape of {w.shape[0]} rows and {w.shape[1]} columns")
    # print(f"b has a shape of {b.shape[0]} rows and {b.shape[1]} columns")

    costs = []
    for iteration in range(n_epochs):
        grad, cost = propagation(w, b, X, Y)

        dw = grad['dw']
        db = grad['db']
        # import pdb
        # pdb.set_trace()
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the costs
        if iteration % 10 == 0:
            costs.append(cost)

        # Print the cost every 100 training examples
        if print_cost and iteration % 100 == 0:
            y_pred = np.dot(X, w.T) + b
            accuracy = np.sqrt(np.mean((y_pred - Y)**2))
            display("Cost after iteration %i: %f | rmse after iteration %i: %f" % (iteration, cost, iteration, accuracy))

    params = {"w": w,
              "b": b}

    grads = {"dw": dw,
             "db": db}

    return params, grads, costs

### Prediction

In [8]:
def predict(w, b, X):
  y_predicted = (np.dot(X, params["w"].T) + params["b"])
  return y_predicted

### Model Evaluation

R2 Score

In [9]:
def calculate_r2_score(y_true, y_pred):

  ssr = (np.mean((y_pred - y_true)**2))
  sst = (np.mean((np.mean(y_true) - y_true)**2))

  R2_score = 1.0 - (ssr/sst)

  return R2_score

In [10]:
params, _, _ = train_linear_regression(X_train, Y_train, print_cost= True)

y_predicted = (np.dot(X_test, params["w"].T) + params["b"])

rmse = np.sqrt(np.mean((y_predicted - Y_test)**2))
print(calculate_r2_score(Y_test, y_predicted))
# display("Model R2 score is %f"%R2_score)

# from sklearn.metrics import r2_score
# r2_score(Y_test, y_predicted)

There are 10 features and 353 training examples
W has a shape of 1 rows and 10 columns


'Cost after iteration 0: 142711531.250000 | rmse after iteration 0: 143.257858'

'Cost after iteration 100: 0.424923 | rmse after iteration 100: 72.040547'

'Cost after iteration 200: 0.289734 | rmse after iteration 200: 68.547067'

'Cost after iteration 300: 0.197304 | rmse after iteration 300: 65.855888'

...

'Cost after iteration 600: 0.061726 | rmse after iteration 600: 60.847491'

'Cost after iteration 700: 0.041744 | rmse after iteration 700: 59.809771'

'Cost after iteration 800: 0.028166 | rmse after iteration 800: 58.963052'

'Cost after iteration 900: 0.018955 | rmse after iteration 900: 58.261154'

0.40040972315107415


# Logistic Regression

### Overview of the Problem Statement

In 2020, there were 2.3 million women diagnosed with breast cancer and 685 000 deaths globally. With this, detecting breast cancer and determining whether its tumour is malignant or benign is a key challenge. In this tutorial, we will build a Logistic Regression classifier that learns to diagnose breast cancer as benign or malignant based on its tumour attributes. To train our model, we will leverage the copy of the Breast Cancer Wisconsin (Diagnostic) dataset provided by `sklearn`. Ten real-valued features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. These features describe the characteristics of each cell nucleus. They are listed as follows:

1.   radius (mean of distances from the centre to points on the perimeter)
2.   texture (standard deviation of gray-scale values)
3.   perimeter
4.   area
5.   smoothness (local variation in radius lengths)
6.   compactness (perimeter^2 / area - 1.0)
7.   concavity (severity of concave portions of the contour)
7.   concave points (number of concave portions of the contour)
8.   symmetry
9.   fractal dimension ("coastline approximation" - 1)

The mean, standard error, and “worst” or largest (mean of the three worst/largest values) of these features were computed for each image, resulting in total *30 features*.

More information about this dataset can be found on the following sites:
*   https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)
*   https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [11]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets

### Getting Familiar with the Data and Notations

*   n ⟶ number of features
*   m ⟶ number of training examples
*   X ⟶ input data of shape (m x n)
*   Y ⟶ target data of shape (m x 1)
*   x(i), y(i) ⟶ ith training example

In [12]:
def load_breast_cancer_dataset():
    '''
    This function loads the dataset.
    It also prints the description of the dataset.

    Returns:
    X - Feature values of the samples.
    Y - Target variable. Takes value either 0(benign) or 1(malignant)
    '''
    data_dict = datasets.load_breast_cancer()
    X, Y = data_dict['data'], data_dict['target']
    Y = Y.reshape(len(Y), 1) # reshape the label vector so that it has a dimesion of m * k

    print("Total number of instances in the dataset: ", X.shape[0])
    print(f"There are {X.shape[1]} features as follows: ", data_dict.feature_names)
    return X, Y

In [13]:
X, Y = load_breast_cancer_dataset()

# preprocess the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# split into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10)

Total number of instances in the dataset:  569
There are 30 features as follows:  ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


<hr>

> [Activity 1: 6 marks]: Set the values for:
- m_train → number of training instances
- m_test → number of test instances
- n → number of features

> **Expected Output:**

>|||
| ----------- | ----------- |
| m_train | 512 |
| m_test | 57 |
| n | 30 |

<hr>

In [14]:

# # START YOUR CODE HERE

m_train, n = X_train.shape
m_test = X_test.shape[0]


# # END YOUR CODE HERE

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Number of features for each instance: num_features = " + str(n))

print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(Y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(Y_test.shape))

Number of training examples: m_train = 512
Number of testing examples: m_test = 57
Number of features for each instance: num_features = 30
train_set_x shape: (512, 30)
train_set_y shape: (512, 1)
test_set_x shape: (57, 30)
test_set_y shape: (57, 1)


### General Architecture of the Logistic Regression

The key steps in the implementation of logistic regression are:

1. Initialise Model Parameters

2. Loop:

  a. Perform forward propagation

  b. Compute cost and gradients

  c. Update model parameters


We will build all these steps seperately and finally integrate them to define logistic regression model.

### 1. Initialise Model Parameters

In [15]:
def initialize_dimensions(X, Y):
    '''
    This function initializes the values of m, n and k
    m -> number of training instances
    n -> number of features
    k -> number of output labels (for binary classification, k= 1)

    '''
    m, n = X.shape
    k = Y.shape[1]
    return m, n, k

In [16]:
def initialize_weights_with_zeros(dim):
    '''
    This function initializes the weights vector and bias vector

    The genaral formula for Weights matrix and bias vector is -
    W -> number of output units x number of input features
    b -> 1 x number of output units

    '''
    W = np.zeros(shape = (dim))
    b = 0
    return W, b

Computing the sigmoid gradients:
* a) dW
* b) db

Updating weights using the gradient descent equation

### 2.a. Forward Propagation:

The mathematical expression for forward propagation of logistic regression includes two steps:

Step 1. Compute preactivations (Z) as:

\begin{equation}
Z = XW^T +b
\end{equation}

Step 2. Compute sigmoid activations ($\hat{Y}$) for actual prediction:

The sigmoid activation function limits the output to range between 0 and 1. The mathematical equation of a sigmoid activation function is given by:

\begin{equation}
\hat{Y} = sigmoid(Z) = \frac{1}{1 + e^{-(Z)}}
\end{equation}

<hr>

> [Activity 2: 4 marks]: Use the equations above to implement `preactivation()` and then perform sigmoid transformation using `sigmoid()` for predicting $\hat{Y}$.

<hr>

In [17]:
def preactivation(X, w, b):
    '''
    This function computes the preactivation (Z) using weight and bias.
    '''

    # # START YOUR CODE HERE

    z = np.dot(X, w.T) + b

    # # END YOUR CODE HERE
    return z

def sigmoid(z):
    '''
    This function returns the sigmoid of vector (numpy array).
    '''

    # # START YOUR CODE HERE

    a = 1 / (1 + np.exp(-z))

    # # END YOUR CODE HERE
    return a

Testing the implementation


In [18]:
x1 =  np.array([[4.4, 0.9, 2.9], [3.2, 0.7, 2.6]]) # (m * n)

w1 = np.array([[0.1, 0.2, 0.3]]) # (k * n)
b1 =  0 # (1 * k)

print ("preactivation for x1, w1, b1 = " + str(preactivation(x1, w1, b1)))
print ("sigmoid([[1.49], [1.24]]) = " + str(sigmoid(np.array([[1.49], [1.24]]))))

preactivation for x1, w1, b1 = [[1.49]
 [1.24]]
sigmoid([[1.49], [1.24]]) = [[0.81607827]
 [0.77556401]]


**Expected Output:**

|||
|---|---|
preactivation for x1, w1, b1  | [[1.49] [1.24]]
sigmoid([[1.49], [1.24]]) | [[0.81607827] [0.77556401]]

### 2.b. Compute Cost \/Loss and Gradients.

The mathematical expression of the logistic regression cost function is given by:

\begin{equation}
cost = - \ \frac{1}{m} \sum_{i =1}^{m} y_i \log(\hat{Y}_i) + \ (1-y_i) \log(1-\hat{Y}_i)
\end{equation}

It is important to note that, cost/error/loss is a scaler which we would like to minimize using gradient descent. The gradients of the cost function are derived using the chain rule.

The final equations of the gradients are:

\begin{align}
\\
\frac{\partial L}{\partial W}  = \frac{1}{m}(\hat{Y} - Y)^T X
\\
\frac{\partial L}{\partial b} = \frac{1}{m} \sum_{i =1}^{m} (\hat{Y} - Y)
\end{align}

<hr>

> [Activity 3: 20 marks]: Use the equations above to compute cost using `compute_sigmoid_cost()`. Then implement `compute_sigmoid_gradients()` to compute its gradients *dw* and *db*.

<hr>

In [19]:
def compute_sigmoid_cost(Y, A, m):
    # # START YOUR CODE HERE

    cost = -(1 / m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))

    # # END YOUR CODE HERE
    return cost

def compute_sigmoid_gradients(X, A, Y, m):
    # # START YOUR CODE HERE

    dw = (1 / m) * np.dot((A - Y).T, X)
    db = (1 / m) * np.sum(A - Y)

    # # END YOUR CODE HERE
    return dw, db

def propagation(w, b, X, Y):
    # # FORWARD PROPAGATION (FROM X TO COST)
    # START YOUR CODE HERE

    prop = preactivation(X, w, b)
    activations = sigmoid(prop)
    cost = compute_sigmoid_cost(Y, activations, X.shape[0])

    # END YOUR CODE HERE
    # # BACKWARD PROPAGATION (TO FIND GRAD)
    dw, db = compute_sigmoid_gradients(X, activations, Y, X.shape[0])

    cost = np.squeeze(cost)

    grads = {"dw": dw,
             "db": db}

    return grads, cost, activations


Testing the implementation


In [20]:
y1 = np.array([[1], [0]])
a1 =  np.array([[0.81607827], [0.77556401]]) # (m * 1)
m = 2

w1 = np.array([[0.1, 0.2, 0.3]]) # (k * n)
b1 =  0 # (1 * k)


print ("compute_sigmoid_cost = " + str(compute_sigmoid_cost(y1, a1, m)))

dw1, db1 = compute_sigmoid_gradients(x1, a1, y1, m)
print ("compute_sigmoid_gradients dw1 = "+str(dw1)+" db1 = "+str(db1))

compute_sigmoid_cost = 0.8487048722248669
compute_sigmoid_gradients dw1 = [[0.83627461 0.18868262 0.7415467 ]] db1 = 0.29582114


**Expected Output:**

|||
|---|---|
compute_sigmoid_cost | [0.84870487]
|||
compute_sigmoid_gradients
dw1| [[0.83627461 0.18868262 0.7415467 ]]
db | 0.29582114


### 2.c. Updating Model Parameters

The weights can be updated using gradient descent equations as follows -
\begin{align}
\\
W = W - (learning\_rate * \frac{\partial L}{\partial W})
\\
b = b - (learning\_rate * \frac{\partial L}{\partial b})
\end{align}

### Assembling Logistic Regression Model

<hr>

> [Activity 4: 20 marks]: Now that we have implemented the helper functions above, it's time to use them and stitch our logistic regression model together.

<hr>

In [21]:
def train_logistic_regression(X, Y, learning_rate = 0.02, n_epochs = 2000, print_cost = True):

    m,n,k  = initialize_dimensions(X, Y)

    # # initialise weights and biases with zeros
    # # START CODE HERE
    print(f"There are {n} features and {m} training examples")
    w, b = initialize_weights_with_zeros((1, n))
    print(f"W has a shape of {w.shape[0]} rows and {w.shape[1]} columns")
    # # END CODE HERE

    costs = []

    for iteration in range(n_epochs):
        # # perform one epoch to compute weight gradients and cost
        # # START CODE HERE
        grad, cost, activations = propagation(w, b, X, Y)
        # # END CODE HERE

        dw = grad['dw']
        db = grad['db']

        # # parameter update rule
        # # START CODE HERE

        w = w - learning_rate * dw
        b = b - learning_rate * db

        # # END CODE HERE

        # Record the costs
        if iteration % 10 == 0:
            costs.append(cost)

        # Write your code to Print the cost every 100 training examples

        if iteration % 100 == 0:
            y_pred = (activations >= 0.5).astype(int)
            accuracy = np.mean(y_pred == Y) * 100
            print(f"Cost after iteration {iteration}: {cost:.6f} | accuracy after iteration {iteration}: {accuracy}%")

    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    return params, grads, costs

In [22]:
kl = train_logistic_regression(X_train, Y_train, print_cost= True)

There are 30 features and 512 training examples
W has a shape of 1 rows and 30 columns
Cost after iteration 0: 0.693147 | accuracy after iteration 0: 63.0859375%
Cost after iteration 100: 0.640064 | accuracy after iteration 100: 75.9765625%
Cost after iteration 200: 0.596971 | accuracy after iteration 200: 84.1796875%
Cost after iteration 300: 0.560385 | accuracy after iteration 300: 86.71875%
Cost after iteration 400: 0.529048 | accuracy after iteration 400: 88.28125%
Cost after iteration 500: 0.501996 | accuracy after iteration 500: 89.0625%
Cost after iteration 600: 0.478453 | accuracy after iteration 600: 89.84375%
Cost after iteration 700: 0.457802 | accuracy after iteration 700: 90.625%
Cost after iteration 800: 0.439553 | accuracy after iteration 800: 91.015625%
Cost after iteration 900: 0.423314 | accuracy after iteration 900: 91.2109375%
Cost after iteration 1000: 0.408770 | accuracy after iteration 1000: 91.6015625%
Cost after iteration 1100: 0.395667 | accuracy after iterati

### Prediction

In [23]:
def predict(lr_model, X, threshold = 0.5):

  '''
  This function uses the learned parameters of the
  model to classify the test data.

  Sigmoid activation outputs the probability that the tumor for training
  instance x is malignant i.e. belongs to class 1.

  For this demonstration, we will consider all probabilities ≥ 0.5 as
  belonging to class 1 and all probabilities < 0 = class 0.
  '''

  w, b = lr_model[0]["w"],lr_model[0]["b"]
  y_predicted = sigmoid(np.dot(X, w.T) + b)
  Y_pred = np.array([1 if x>threshold else 0 for x in y_predicted])
  Y_pred = np.expand_dims(Y_pred, axis=1)
  return Y_pred

In [24]:
Y_pred = predict(kl, X_test)

### Evaluating Model Performance

In [25]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

#START YOUR CODE HERE

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)

#END YOUR CODE HERE

print(f"Precision: ", precision," Accuracy: ", accuracy,"Recall: ", recall)
print("Confusion Matrix")
print(conf_matrix)

Precision:  0.918918918918919  Accuracy:  0.9473684210526315 Recall:  1.0
Confusion Matrix
[[20  3]
 [ 0 34]]
